In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep

In [2]:
urls_dt = pd.read_csv('hotpepper_kokubuncho.csv')
urls_dt

,Unnamed: 0,name,food_link,drink_link
0,0,隠れ家個室居酒屋 牛どき 仙台国分町,https://www.hotpepper.jp/strJ001202101/food/,https://www.hotpepper.jp/strJ001202101/drink/
1,1,【3月下旬リニューアル】個室居酒屋 粋～いなせ～国分町店,https://www.hotpepper.jp/strJ001170770/food/,https://www.hotpepper.jp/strJ001170770/drink/
2,2,【4/3 OPEN】東北名物×日本酒堪能 個室完備居酒屋 仙台酒場 国分町店,https://www.hotpepper.jp/strJ001217072/food/,https://www.hotpepper.jp/strJ001217072/drink/
3,3,チーズダッカルビ食べ放題 完全個室 肉バル×チーズ うしどき 仙台国分町,https://www.hotpepper.jp/strJ001193888/food/,https://www.hotpepper.jp/strJ001193888/drink/
4,4,ぼんてん漁港勾当台店,https://www.hotpepper.jp/strJ001135316/food/,https://www.hotpepper.jp/strJ001135316/drink/
5,5,焼き鳥と桜肉と日本酒 鶏櫻,https://www.hotpepper.jp/strJ001171389/food/,https://www.hotpepper.jp/strJ001171389/drink/
6,6,和み酒房 蛍火 ほたるび 虎横店 個室 牛タン 宴会 日本酒 飲み放題,https://www.hotpepper.jp/strJ001195130/food/,https://www.hotpepper.jp/strJ001195130/drink/
7,7,鍋と肉居酒屋 牛タン 焼き鳥 是空 仙台国分町,https://www.hotpepper.jp/strJ001054839/food/,https://www.hotpepper.jp/strJ001054839/drink/
8,8,発酵の里 侘びいち 国分町 居酒屋 完全個室 せり鍋,https://www.hotpepper.jp/strJ001118722/food/,https://www.hotpepper.jp/strJ001118722/drink/
9,9,【贔屓】,https://www.hotpepper.jp/strJ001168035/food/,https://www.hotpepper.jp/strJ001168035/drink/


In [3]:
index=0

In [4]:
urls_dt.at[index,'food_link']

'https://www.hotpepper.jp/strJ001202101/food/'

In [5]:
# import requests
url = urls_dt.at[index,'food_link']
response = requests.get(url)
print("response.status_code:"+str(response.status_code))

response.status_code:200


In [6]:
# from bs4 import BeautifulSoup
bs = BeautifulSoup(response.content,"lxml")
print("bs ok.")

bs ok.


In [7]:
class Shop_info:
    def __init__(self):
        self.name = ''
        self.location = {'lat':[],'lng':[]}
        self.menu = {'kodawari':{'name':[],'text':[]},
                     'osusume':{'dish':[],'text':[],'price':[]},
                     'main':{'category':[],
                             'menu':{'dish':[],
                                     'price':[],
                                     'icon':[],
                                     'catch':[]
                                    }
                            }
               }
        self.drink = {'osusume':{'name':[]}}

In [8]:
shop = []
shop.append(Shop_info())
for d in bs.select('#mainContentsWrapSecond > div.shopInner > div.locator.photoExtra > div'):
    if len(d.text) > 5:
        shop[0].menu['osusume']['dish'].append(d.select('h3')[0].text)
        shop[0].menu['osusume']['text'].append(d.select_one('p:nth-child(3)').text)
        shop[0].menu['osusume']['price'].append(d.select('p.price')[0].text)
shop[0].menu['osusume']

{'dish': ['漁港直送鮮魚やこだわり和食でご宴会に彩を◎',
  '【半額!!】スタンダード飲み放題が半額の1980円→980円に!!お食事は単品でご注文頂けます♪',
  '最上級のお肉料理をリーズナブルにご提供いたします◎'],
 'text': ['甘み豊かな鯛とTVでも紹介された和牛…思わずチョイスに迷う贅沢コースをご用意。最大3時間の飲み放題がついて2480円から！その他にも季節の旬魚料理など、当店自慢の絶品料理が彩ります。料理長が丁寧に仕上げる料理の数々をご賞味下さい。',
  '毎日使えるお得なクーポン複数ございます◎一番人気はお食事は単品でご注文いただける飲み放題半額の特典♪ビールやカクテル、日本酒など約７０種類からご注文頂けます!!二次会や三次会にも最適、当日予約も大歓迎♪',
  '鮮度、産地、品質、味に徹底的にこだわった元焼肉屋の仕入れルートでおいしいお肉をご用意しています！！こだわりの焼き方で焼く時間と同じ時間肉を休ませることで肉汁を閉じ込めます。うしどきでしか味わえないお肉料理を是非！！'],
 'price': ['2,480円より', '1980円→半額の980円', '1,280円→680円']}

In [9]:
dish_html_title = bs.select('#dish')[0].text
dish_html=bs.select('#mainContentsWrapSecond > div.shopInner > div.dish')[0]
count = 0
for i,d in enumerate(dish_html.select(".dish>div.locator")):
    if len(d.text) > 20:
#       category
        count += 1
        if dish_html.select('#category'+str(count)):
            shop[0].menu['main']['category'].append(dish_html.select('#category'+str(count))[0].text)
        else:
            shop[0].menu['main']['category'].append("category"+str(count))
#       menu
        for j,m in enumerate(d.select('div.menu')):
            if m.select('h4'):
                shop[0].menu['main']['menu']['dish'].append(m.select('h4')[0].text)
            else:
                shop[0].menu['main']['menu']['dish'].append("")
            if m.select('p.price'):
                shop[0].menu['main']['menu']['price'].append(m.select('p.price')[0].text)
            else:
                shop[0].menu['main']['menu']['price'].append("Ask")
            if len(list(m.next_siblings))>1:
                shop[0].menu['main']['menu']['icon'].append(str(list(map(lambda x: x.get('alt'),list(m.next_siblings)[1].select('img')))))
            else:
                shop[0].menu['main']['menu']['icon'].append("")
            if len(list(m.next_siblings))>3:
                shop[0].menu['main']['menu']['catch'].append(list(m.next_siblings)[3].text)
            else:
                shop[0].menu['main']['menu']['catch'].append("")
            
shop[0].menu['main']

{'category': ['category1',
  '■逸品料理',
  '■肉',
  '■即菜',
  '■冷菜',
  '■サラダ',
  '■焼き物',
  '■揚げ物',
  '■締めもの',
  '■デザート'],
 'menu': {'dish': ['料理長自慢のチーズタッカルビがなんと1480円で食べ放題！',
   '『うしどきコース』10品3H飲み放題付き⇒3980円',
   'ボリューム満点で大満足のコスパプランが気軽に楽しめる',
   '韓国で話題沸騰中！チーズタッカルビ',
   '牛タン焼き',
   'タンしゃぶ',
   '牛ハラミのステーキ~季節野菜を添えて~',
   '牛焼肉の鉄板焼き',
   '厚切りベーコンステーキ',
   '豚バラ肉のソテー~マスタードソース~',
   '若鶏の香草焼き',
   'ソーセージの盛り合わせ',
   '熟成牛と肉の盛り合わせ',
   '肉塩キャベツ',
   '季節野菜のピクルス',
   'フレッシュトマトのカプレーゼ',
   '鴨燻製のチーズ炙り',
   '鴨肉のカルパッチョ',
   '鮮魚のカルパッチョ',
   '桜肉のユッケ',
   'チーズの盛り合わせ',
   '生ハムの盛り合わせ',
   '半熟玉子のシーザーサラダ',
   '豚肉とアボカドのグリーンサラダ',
   '肉味噌のバーニャカウダー',
   '肉屋のポテトサラダ',
   '肉味噌のだし巻き玉子',
   'ジューシー肉餃子',
   '肉ナスの鉄板焼き',
   '豪快若鶏の一枚揚げ',
   'フライドポテト',
   '熟成醤油仕込みのフライドキチン',
   'わか鳥の唐揚げ',
   'チキン南蛮~特製タルタルソース~',
   'もっちりチーズ揚げ',
   'ピッツァマルゲリータ',
   'クワトロフォルマッジ',
   '本場仕込みのタコライス',
   'バニラアイス',
   'フォンダンショコラ',
   'クリームブリュレ',
   'ベイクドチーズケーキ'],
  'price': ['期間限定価格',
   '至極な宴会プラン',
   '病みつきの美味しさ',
   '1,280円(税別)',
   '1,280円(税別)',
   '1,680円(

In [60]:
menu_dt = pd.DataFrame.from_dict(shop[0].menu['main']['menu'])
menu_dt

,dish,price,icon,catch
0,料理長自慢のチーズタッカルビがなんと1480円で食べ放題！,期間限定価格,['大人気'],都内を中心に今話題のチーズタッカルビ。うしどきではそのチーズタッカルビが1480円で食べ放題...
1,『うしどきコース』10品3H飲み放題付き⇒3980円,至極な宴会プラン,['オリジナル'],どれも澄みきった素材の品々をまとめてコース提案させていただきました。驚きの飲み放題付きの特別...
2,ボリューム満点で大満足のコスパプランが気軽に楽しめる,病みつきの美味しさ,['限定'],気仙沼海のオーソドックスなまぐろのほか、旬のものをお入れしておりますので、お刺身は身が引き締...
3,韓国で話題沸騰中！チーズタッカルビ,"1,280円(税別)",['大人気'],
4,牛タン焼き,"1,280円(税別)",['定番'],
5,タンしゃぶ,"1,680円(税別)",['絶品'],
6,牛ハラミのステーキ~季節野菜を添えて~,"1,380円(税別)",,
7,牛焼肉の鉄板焼き,"1,280円(税別)",,
8,厚切りベーコンステーキ,"1,280円(税別)",,
9,豚バラ肉のソテー~マスタードソース~,"1,230円(税別)",,


In [10]:
urls_dt.at[index,'drink_link']

'https://www.hotpepper.jp/strJ001202101/drink/'

In [128]:
# import requests
url = urls_dt.at[index,'drink_link']
response = requests.get(url)
print("response.status_code:"+str(response.status_code))

response.status_code:200


In [129]:
# from bs4 import BeautifulSoup
bs = BeautifulSoup(response.content,"lxml")
print("bs ok.")

bs ok.


In [130]:
appeal_elem = bs.select('div.appeal')[0]
appeal_elem

<div class="boxedItem appeal">
<!-- comment for IE bug (Don't delete!) -->
<h2>ドリンクのこだわり</h2>
<div class="locator shopAboutList">
<div class="column4B">
<h3>充実のドリンク！ノンアルコールドリンクも豊富</h3>
<div class="locator">
<div class="columnPssA">
<p class="photoSS slideShow">
<a class="imagethickbox" href="https://imgfp.hotp.jp/IMGH/30/38/P024383038/P024383038_480.jpg" rel="drinkChoosy" title="充実のドリンク！ノンアルコールドリンクも豊富">
<img alt="充実のドリンク！ノンアルコールドリンクも豊富" src="https://imgfp.hotp.jp/IMGH/30/38/P024383038/P024383038_78.jpg"/></a>
</p>
</div>
<div class="columnPssB">
<p>豊富なドリンクメニューは量だけではなく質にこだわりました。ハイボールや酎ハイサワーは強炭酸のソーダを使用しております。</p>
</div>
</div>
</div>
<div class="column4B">
<h3>ワインやプレミアムなシャンパンもご用意しております。</h3>
<div class="locator">
<div class="columnPssA">
<p class="photoSS slideShow">
<a class="imagethickbox" href="https://imgfp.hotp.jp/IMGH/30/45/P024383045/P024383045_480.jpg" rel="drinkChoosy" title="ワインやプレミアムなシャンパンもご用意しております。">
<img alt="ワインやプレミアムなシャンパンもご用意しております。" src="https://imgfp.hotp.jp/IMGH/30/45

In [131]:
appeal = { 'name':[] , 'text':[] }
if appeal_elem:
    for a in appeal_elem.select('h3'):
        appeal['name'].append(a.text)
        appeal['text'].append(list(a.next_siblings)[1].select('.columnPssB')[0].text)
        appeal['text'][-1]=appeal['text'][-1].replace('\n','')
    appeal    

In [132]:
osusume_elem = bs.select('div.locator.photoExtra')[0]
osusume_elem

<div class="locator photoExtra">
<div class="columnPlex">
<p class="photoL slideShow">
<a class="imagethickbox" href="https://imgfp.hotp.jp/IMGH/20/69/P025072069/P025072069_480.jpg" rel="firstDrink" title="プレミアムモルツ、ハイボール、酎ハイ、果実酒、ノンアルコールも充実！な【プレミアム飲み放題】">
<span><img alt="牛どき 仙台国分町のおすすめドリンク1" src="https://imgfp.hotp.jp/IMGH/20/69/P025072069/P025072069_238.jpg"/></span>
</a>
</p>
<h3>プレミアムモルツ、ハイボール、酎ハイ、果実酒、ノンアルコールも充実！な【プレミアム飲み放題】</h3>
<p></p>
<p class="price">お1人様1500円</p>
</div>
</div>

In [133]:
osusume = {'name':[],'price':[]}
for o in osusume_elem.select('h3'):
    osusume['name'].append(o.text)
    osusume['price'].append(o.parent.select('p.price')[0].text)
osusume

{'name': ['プレミアムモルツ、ハイボール、酎ハイ、果実酒、ノンアルコールも充実！な【プレミアム飲み放題】'],
 'price': ['お1人様1500円']}

In [134]:
clist

[<h3>サントリープレミアムモルツ</h3>, <dl class="price">
 <dt>ジョッキ</dt>
 <dd>630円(税別)</dd>
 </dl>, <h3>シャンディーガフ</h3>, <dl class="price">
 <dt></dt>
 <dd>580円(税別)</dd>
 </dl>, <h3>オールフリー</h3>, <dl class="price">
 <dt>ノンアルコール(小瓶)</dt>
 <dd>480円(税別)</dd>
 </dl>, <h2 id="category2"><span class="shopTitleNoLink">■サワー</span></h2>, <h3>ウーロンハイ</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>緑茶ハイ</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>ジャスミンハイ</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>ライムサワー</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>アセロラサワー</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>レモンサワー</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>グレープフルーツサワー</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h3>オレンジサワー</h3>, <dl class="price">
 <dt></dt>
 <dd>450円(税別)</dd>
 </dl>, <h2 id="category3"><span class="shopTitleNoLink">■角ハイボール</span></h2>, <h3>ハイボール</h3>, <d

In [135]:
drink = { 'category':[],'drink':[]}
drink_tmp = {'name':[] , 'price':[]}
clist = list(bs.select('#category1')[0].next_siblings)
clist = list(filter(lambda x: ('\n' != x) ,clist))
drink['category'].append(bs.select('#category1')[0].text)
for i,c in enumerate(clist):
    if 'Comment' in str(type(c)):
        continue
    if 'h3' in str(c):
        drink_tmp['name'].append(c.text)
    if 'dl' in str(c):
        drink_tmp['price'].append(c.select('dd')[0].text)
    if 'h2' in str(c):
        drink['drink'].append(drink_tmp)
        drink['category'].append(c.text)
        drink_tmp =  {'name':[] , 'price':[]}
drink['drink'].append(drink_tmp)
         
drink        
        

{'category': ['■ビール',
  '■サワー',
  '■角ハイボール',
  '■カクテル',
  '■日本酒',
  '■梅酒',
  '■焼酎',
  '■ソフトドリンク',
  '■ワイン',
  '■サングリア',
  '牛どき 仙台国分町 飲み放題メニュー'],
 'drink': [{'name': ['サントリープレミアムモルツ', 'シャンディーガフ', 'オールフリー'],
   'price': ['630円(税別)', '580円(税別)', '480円(税別)']},
  {'name': ['ウーロンハイ',
    '緑茶ハイ',
    'ジャスミンハイ',
    'ライムサワー',
    'アセロラサワー',
    'レモンサワー',
    'グレープフルーツサワー',
    'オレンジサワー'],
   'price': ['450円(税別)',
    '450円(税別)',
    '450円(税別)',
    '450円(税別)',
    '450円(税別)',
    '450円(税別)',
    '450円(税別)',
    '450円(税別)']},
  {'name': ['ハイボール', 'コークハイボール', 'ジンジャーハイボール'],
   'price': ['530円(税別)', '530円(税別)', '530円(税別)']},
  {'name': ['カシス', 'ライチ', 'ピーチ', 'ジン'],
   'price': ['500円(税別)', '500円(税別)', '500円(税別)', '500円(税別)']},
  {'name': ['一ノ蔵\u3000無鑑査\u3000辛口'], 'price': ['580円(税別)']},
  {'name': ['南高梅梅酒'], 'price': ['480円(税別)']},
  {'name': ['黒霧島 20度 芋', 'なんこ 25度 芋', '一刻者 芋', 'わんこ 麦', '鍛高譚 紫蘇'],
   'price': ['480円(税別)', '540円(税別)', '580円(税別)', '540円(税別)', '540円(税別)']},
  {'name': ['グレープフルーツジュース'

In [60]:
menu_dt = pd.DataFrame.from_dict(shop[0].menu['main']['menu'])
menu_dt

,dish,price,icon,catch
0,料理長自慢のチーズタッカルビがなんと1480円で食べ放題！,期間限定価格,['大人気'],都内を中心に今話題のチーズタッカルビ。うしどきではそのチーズタッカルビが1480円で食べ放題...
1,『うしどきコース』10品3H飲み放題付き⇒3980円,至極な宴会プラン,['オリジナル'],どれも澄みきった素材の品々をまとめてコース提案させていただきました。驚きの飲み放題付きの特別...
2,ボリューム満点で大満足のコスパプランが気軽に楽しめる,病みつきの美味しさ,['限定'],気仙沼海のオーソドックスなまぐろのほか、旬のものをお入れしておりますので、お刺身は身が引き締...
3,韓国で話題沸騰中！チーズタッカルビ,"1,280円(税別)",['大人気'],
4,牛タン焼き,"1,280円(税別)",['定番'],
5,タンしゃぶ,"1,680円(税別)",['絶品'],
6,牛ハラミのステーキ~季節野菜を添えて~,"1,380円(税別)",,
7,牛焼肉の鉄板焼き,"1,280円(税別)",,
8,厚切りベーコンステーキ,"1,280円(税別)",,
9,豚バラ肉のソテー~マスタードソース~,"1,230円(税別)",,
